In [ ]:
def complete_word(language_model, text, final_word, no_unk=True, decoder=decode_spec_tokens, temperature=1.):
    
    language_model.model.reset()
    language_model.model.eval()
    numericalize = language_model.dls.train_ds.numericalize
    idxs = idxs_all = language_model.dls.test_dl([text]).items[0].to(language_model.dls.device)
    if no_unk: unk_idx = language_model.dls.vocab.index(UNK)
        
    with torch.no_grad():
        preds,_ = language_model.get_preds(dl=[(idxs[None],)])
        res = preds[0][-1]
        sorted_indices = torch.argsort(res, descending=True)
        for idx in sorted_indices:
            if idx not in ['xxbos', 'xxpad', 'xxunk', 'xxmaj']:
                if numericalize.vocab[idx].lower().startswith(final_word):
                    return numericalize.vocab[idx][len(final_word):]
        return ""

In [44]:
from transformers import AutoTokenizer, AutoModelWithLMHead

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [1]:
from fastai.text.all import *

In [3]:
language_model = load_learner('../5epochs_imdb_lm.pkl')

In [40]:
text = "this movie speaks about the atrocities committed by the east india and and the unkadsj alsdjaksljd aljdlascorporation in the region of tehran. I think that the main issue here is that"

In [118]:
language_model.predict??

In [42]:
import fastai
print(fastai.__version__)
import fastcore
print(fastcore.__version__)

2.0.19
1.3.2


In [45]:

tokenizer = AutoTokenizer.from_pretrained("gpt2")
neutral_model = AutoModelWithLMHead.from_pretrained("lvwerra/gpt2-imdb")
positive_model = AutoModelWithLMHead.from_pretrained("/home/advaitmb/notebooks/projects/PreDiction/nbs/gpt2-imdb-positive-sentiment")
negative_model = AutoModelWithLMHead.from_pretrained("/home/advaitmb/notebooks/projects/PreDiction/nbs/gpt2-imdb-negative-sentiment")
neutral_model.to('cuda')
positive_model.to('cuda')
negative_model.to('cuda')

/anaconda/envs/fastai2/lib/python3.8/site-packages/transformers/modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [116]:
def complete_word_transformer(language_model, tokenizer, text, final_word):
#     text = text.strip()
#     final_word = final_word.strip()
    if len(text) == 0:
        return ''
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    logits = language_model(t, return_dict=True).logits[0]
    sorted_indices = torch.argsort(logits, descending=True)
    print(sorted_indices.size())
    for tk_idx in sorted_indices:
        word = tokenizer.decode(tk_idx.cpu().numpy())
        print(word)
        if word.lower().startswith(final_word):
            return word[len(final_word):]
    return ""

In [117]:
complete_word_transformer(neutral_model, tokenizer, 'this is', 't')

torch.Size([2, 50257])
,. is and- to's in the was of/ a: " ( has I that) for are one on can" will as time or - would it? at / with this have!' not had could story by you but from<|endoftext|>; show all year may were way
 movie should be... he does film). so version really '. The only like_." we justs makes isn very last if people work also part < more about first which do much they there kind type point," made doesn because out no little did group when who), being =, shows world an end set day might] two my -- other> even good here( : thing class life get man case up new 2 & family line AI name character use long must me what too person seems 1 means most now years aftera [... action game old same great post many over< make where bad number piece need than into anyThe series gets special don big his – book page back your section picture episode 3 idea bit well place some still view takes site style got then used team * far level their before ever full looks C",= #". stuff screen comes 

''